# Uplift Modeling with EconML using MovieLens 1M
This notebook downloads MovieLens 1M data, simulates treatment and renewal outcomes, introduces missing data, imputes values, and trains S-, T-, and X-Learners using different base learners.

In [2]:
!pip uninstall  econml scikit-learn pandas numpy

Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/scikit_learn-1.6.1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/scikit_learn.libs/libgomp-a34b3233.so.1.0.0
    /usr/local/lib/python3.11/dist-packages/sklearn/*
Proceed (Y/n)? y
  Successfully uninstalled scikit-learn-1.6.1
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/pandas-2.2.2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/pandas/*
Proceed (Y/n)? y
  Successfully uninstalled pandas-2.2.2
Found existing installation: numpy 2.0.2
y
Uninstalling numpy-2.0.2:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/numpy-config
    /usr/local/lib/python3.11/dist-packages/numpy-2.0.2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1-0352e75f.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/l

In [3]:
!pip  install --no-cache-dir  econml scikit-learn pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 123.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 147.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 212.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 154.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 184.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 317.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.3/147.3 kB 260.1 MB/s eta 0:00:00
  Attempting uninstall: slicer
    Found existing installation: slicer 0.0.8
    Uninstalling slicer-0.0.8:
      Successfully uninstalled slicer-0.0.8
  Attempting uninstall: shap
    Found existing installation: shap 0.47.2
    Uninstalling shap-0.47.2:
      Successfully unin

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from econml.metalearners import SLearner, TLearner, XLearner

In [2]:
# Download and extract MovieLens 1M dataset
!pip install wget
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip -o ml-1m.zip -d ml-1m

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=453bb6b65565fe5fa4ef2a102a6e9607d1b368c099a29f9feb1bf986dac5de95
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget
--2025-05-19 14:24:43--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  10.7MB/s    in 0.5s    

2025-05-19 14:24:44 (10.7 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/ml-1m/
  inflating: ml-1m/ml-1m/movies.dat  
  inflating: ml-1m/ml-1m/ratings.dat  
  inflating: ml-1m/ml-1m/README      
  inflating: ml-1m

In [3]:
# Download and extract MovieLens 1M dataset
# The wget and unzip commands appear to be working correctly based on your output.
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip -o ml-1m.zip -d ml-1m

# Add checks to verify if the directory and file exist
import os

# Correct the path to reflect the nested directory structure
if os.path.exists('ml-1m/ml-1m/ratings.dat'):
    print("ml-1m/ml-1m/ratings.dat found. Proceeding to load data.")
else:
    print("Error: ml-1m/ml-1m/ratings.dat not found. Please check the extraction path.")
    # If the file is still not found after correcting the path, there might be
    # a deeper issue with the unzip process or disk.
    # import sys
    # sys.exit(1) # Uncomment to exit the notebook execution if the file is not found

--2025-05-19 14:24:44--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.1’

ml-1m.zip.1         100%[===================>]   5.64M  10.6MB/s    in 0.5s    

2025-05-19 14:24:45 (10.6 MB/s) - ‘ml-1m.zip.1’ saved [5917549/5917549]

Archive:  ml-1m.zip
  inflating: ml-1m/ml-1m/movies.dat  
  inflating: ml-1m/ml-1m/ratings.dat  
  inflating: ml-1m/ml-1m/README      
  inflating: ml-1m/ml-1m/users.dat   
ml-1m/ml-1m/ratings.dat found. Proceeding to load data.


In [4]:
# Load data
ratings = pd.read_csv('ml-1m//ml-1m/ratings.dat', sep='::', engine='python',
                      names=['UserID', 'MovieID', 'Rating', 'Timestamp'])
users = pd.read_csv('ml-1m/ml-1m/users.dat', sep='::', engine='python',
                    names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'])
# Specify the encoding as 'latin-1' or 'ISO-8859-1' for the movies.dat file
#movies = pd.read_csv('ml-1m/ml-1m/movies.dat', sep='::', engine='python',
#                     names=['MovieID', 'Title', 'Genres'], encoding='latin-1')
#df = ratings.merge(users, on='UserID').merge(movies, on='MovieID')

In [5]:
# Specify the encoding as 'latin-1' or 'ISO-8859-1' for the movies.dat file
movies = pd.read_csv('ml-1m/ml-1m/movies.dat', sep='::', engine='python',
                     names=['MovieID', 'Title', 'Genres'], encoding='latin-1')
#df = ratings.merge(users, on='UserID').merge(movies, on='MovieID')

In [6]:
# merge ratings, users, and movies
df = ratings.merge(users, on = 'UserID').merge(movies, on = 'MovieID')
df.sample(10)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres
816732,4904,1150,5,962684596,M,50,15,63121,"Return of Martin Guerre, The (Retour de Martin...",Drama
953122,5755,1307,4,958266013,F,35,2,78744,When Harry Met Sally... (1989),Comedy|Romance
898026,5430,911,5,960073473,F,45,1,26505,Charade (1963),Comedy|Mystery|Romance|Thriller
463987,2860,1293,4,1041542222,M,35,17,20009,Gandhi (1982),Drama
251565,1521,348,4,974745146,F,25,7,08902,Bullets Over Broadway (1994),Comedy
53864,352,191,4,976334645,M,18,4,60115,"Scarlet Letter, The (1995)",Drama
602485,3665,2193,4,973903741,M,25,0,95903,Willow (1988),Action|Adventure|Fantasy
612607,3713,141,5,966270676,F,25,7,22201,"Birdcage, The (1996)",Comedy
526267,3253,318,3,968283912,F,35,4,26505,"Shawshank Redemption, The (1994)",Drama
525917,3249,21,5,968297689,F,25,4,92648,Get Shorty (1995),Action|Comedy|Drama


In [7]:
# Feature creation
np.random.seed(42)
df['WatchTime'] = df['Rating'] * np.random.uniform(15, 30, size=len(df)).astype(int)


In [8]:
df['Timestamp_Date'] = pd.to_datetime(df['Timestamp'], unit='s').dt.strftime('%Y-%m-%d')

In [9]:
df['TenureMonths'] = (df['Timestamp'] - df['Timestamp'].min()) // (60*60*24*30)

In [10]:
#regenerate ages with randome integers between 18-69
df2 = pd.DataFrame()
df2['UserID'] = df['UserID'].drop_duplicates()
df2.head()

,UserID
0,1
53,2
182,3
233,4
254,5


In [11]:
df2['Age'] = np.random.randint(18, 70, df2.shape[0])
df2.head()

,UserID,Age
0,1,37
53,2,30
182,3,61
233,4,23
254,5,34


In [12]:
df_user = df.merge(df2, on = 'UserID', how = 'left')
df_user.sample(5)

,UserID,MovieID,Rating,Timestamp,Gender,Age_x,Occupation,Zip-code,Title,Genres,WatchTime,Timestamp_Date,TenureMonths,Age_y
557666,3425,3809,5,967351972,M,18,20,48135,What About Bob? (1991),Comedy,110,2000-08-27,4,48
85626,558,111,3,976049332,M,35,20,55108,Taxi Driver (1976),Drama|Thriller,51,2000-12-05,7,32
705278,4227,1928,3,965410265,M,25,19,11414-2520,Cimarron (1931),Western,63,2000-08-04,3,19
977162,5888,2324,5,957480090,M,25,20,64114,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama,75,2000-05-04,0,50
987831,5963,296,5,957018501,M,25,15,02140,Pulp Fiction (1994),Crime|Drama,90,2000-04-29,0,18


In [13]:
df_user = df_user.drop('Age_x', axis= 1).rename(columns = {'Age_y':'Age'})
df_user.sample(5)

,UserID,MovieID,Rating,Timestamp,Gender,Occupation,Zip-code,Title,Genres,WatchTime,Timestamp_Date,TenureMonths,Age
675445,4053,2120,3,965493737,M,18,36264,Needful Things (1993),Drama|Horror,84,2000-08-05,3,49
438067,2676,3915,5,973401035,M,20,78731,Girlfight (2000),Drama,110,2000-11-05,6,20
354089,2073,497,5,974665536,F,4,13148,Much Ado About Nothing (1993),Comedy|Romance,130,2000-11-19,6,25
332099,1959,1952,5,976246198,F,13,53092,Midnight Cowboy (1969),Drama,130,2000-12-08,7,60
67834,454,2707,3,976488545,M,20,55092,Arlington Road (1999),Thriller,75,2000-12-10,7,45


In [14]:
df_user.shape

(1000209, 13)

In [15]:
# Feature creation
user_features = df_user.groupby('UserID').agg({
    'WatchTime': 'sum',
    'MovieID': 'nunique',
    'TenureMonths': 'max',
    'Age': 'first',
    'Occupation': 'first'
}).rename(columns={'WatchTime': 'TotalWatchTime', 'MovieID': 'UniqueMovies'})

In [16]:
user_features.sample(5)

,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation
UserID,,,,,
1713,1543,21,6,51,0
1816,5573,62,6,41,9
752,63936,1024,32,60,3
1342,6140,92,6,47,0
3419,10440,120,4,48,1


In [17]:
# Introduce and impute missing data
user_features.loc[user_features.sample(frac=0.1).index, 'TotalWatchTime'] = np.nan
user_features.sample(10)

,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation
UserID,,,,,
1282,1776.0,21,6,36,1
3,NaN,51,8,61,15
4361,11953.0,132,10,52,15
3240,11458.0,154,4,45,4
279,NaN,47,18,40,14
4419,1472.0,20,3,37,12
5909,2049.0,29,0,37,20
2724,17197.0,205,8,49,2
4298,26559.0,285,3,50,14


In [18]:
user_features['TotalWatchTime'].isnull().sum()

604

In [19]:
user_features.loc[user_features.sample(frac=0.1).index, 'TenureMonths'] = np.nan
user_features.sample(15)

,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation
UserID,,,,,
5555,37298.0,548,1.0,68,10
3091,5099.0,64,4.0,22,0
5911,13900.0,171,2.0,48,17
585,11275.0,148,7.0,41,1
1675,40295.0,565,7.0,18,6
1865,26174.0,334,22.0,29,1
442,35801.0,423,NaN,48,1
380,11413.0,138,7.0,55,2
2575,14050.0,169,6.0,24,7


In [20]:
# Introduce and impute missing data
user_features['TotalWatchTime'] =user_features['TotalWatchTime'].fillna(user_features['TotalWatchTime'].median())
user_features['TenureMonths']= user_features['TenureMonths'].fillna(user_features['TenureMonths'].median())

In [22]:
# Simulate treatment and renewal
user_features['treatment'] = np.random.binomial(1, 0.5, size=len(user_features))
engaged = user_features['TotalWatchTime'] > user_features['TotalWatchTime'].median()
base_rate = 0.2
uplift = 0.15 * ((user_features['treatment'] == 1) & engaged).astype(float)
user_features['renewed'] = np.random.binomial(1, base_rate + uplift)
X = user_features[['TenureMonths', 'TotalWatchTime', 'UniqueMovies']]
T = user_features['treatment'].values
Y = user_features['renewed'].values

In [23]:
T.shape, type(T), T

((6040,), numpy.ndarray, array([1, 1, 0, ..., 0, 1, 0]))

In [24]:
# Split data
X_train, X_test, T_train, T_test, Y_train, Y_test = train_test_split(X, T, Y, test_size=0.2, random_state=42)

In [25]:
X_train.head(2)

,TenureMonths,TotalWatchTime,UniqueMovies
UserID,,,
1122,7.0,8331.0,114
4432,6.0,6121.0,68


In [27]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score

In [28]:
# simulate more users
n_new = 20000  # or any number > 6040
simulated_users = user_features.sample(n=n_new, replace=True, random_state=42).reset_index(drop=True)

# Add noise to continuous columns
for col in ['TotalWatchTime', 'TenureMonths', 'Age']:
    noise = np.random.normal(0, user_features[col].std() * 0.05, size=n_new)  # 5% of std deviation
    simulated_users[col] += noise
    simulated_users[col] = simulated_users[col].clip(lower=user_features[col].min(), upper=user_features[col].max())

simulated_users['UserID'] = range(1, n_new+1)
simulated_users = simulated_users.reset_index(drop=True)


In [29]:
#simulated categorical features
# Number of samples already defined
n_samples = simulated_users.shape[0]

# Simulate categorical fields
np.random.seed(42)

simulated_users['device_type'] = np.random.choice(['mobile', 'tablet', 'tv', 'desktop'], size=n_samples, p=[0.4, 0.1, 0.3, 0.2])
simulated_users['subscription_tier'] = np.random.choice(['free', 'basic', 'premium'], size=n_samples, p=[0.2, 0.5, 0.3])
simulated_users['region'] = np.random.choice(['Northeast', 'Midwest', 'South', 'West'], size=n_samples)
simulated_users['has_kids_profile'] = np.random.binomial(1, 0.3, size=n_samples)
simulated_users['promo_eligible'] = np.random.binomial(1, 0.5, size=n_samples)

In [ ]:
simulated_users.head()

,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation,treatment,renewed,UserID,device_type,subscription_tier,region,has_kids_profile,promo_eligible
0,7440.027920,80,7.239655,59.614629,7,1,0,1,mobile,premium,West,1,0
1,1493.701456,23,1.730815,49.277609,7,0,1,2,desktop,free,Northeast,0,1
2,10790.226372,157,1.185210,46.291427,10,0,0,3,tv,basic,Northeast,0,0
3,1755.714462,22,0.953775,39.189768,7,1,0,4,tv,basic,West,1,0
4,27024.149372,361,29.990638,61.782170,6,0,0,5,mobile,basic,South,0,0


In [ ]:
simulated_users['promo_eligible'].value_counts()

,count
promo_eligible,
1,10048
0,9952


In [ ]:
# Simulate A/B Test: assign treatment randomly with 50% probability
np.random.seed(42)  # Ensures reproducibility

n_samples = simulated_users.shape[0]
simulated_users['treatment'] = np.random.binomial(1, 0.5, size=n_samples)

# Check balance
print(simulated_users['treatment'].value_counts(normalize=True))


treatment
0    0.5006
1    0.4994
Name: proportion, dtype: float64


In [ ]:
# Define engagement baseline
engaged = simulated_users['TotalWatchTime'] > simulated_users['TotalWatchTime'].median()

# Baseline renewal rate
base_rate = 0.2 + 0.15 * engaged

# Add uplift: only if treated, and influenced by user features
uplift = (
    0.10 * (simulated_users['treatment'] == 1) *
    (simulated_users['subscription_tier'] == 'basic') +
    0.20 * (simulated_users['treatment'] == 1) *
    (simulated_users['has_kids_profile'] == 1)
)

# Combine into final renewal probability
renewal_prob = base_rate + uplift
renewal_prob = np.clip(renewal_prob, 0.05, 0.95)  # Ensure valid range

# Simulate binary outcome
simulated_users['renewed'] = np.random.binomial(1, renewal_prob)


In [30]:
simulated_users =simulated_users.drop('promo_eligible', axis=1)
#

In [31]:
simulated_users.head(3)

,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation,treatment,renewed,UserID,device_type,subscription_tier,region,has_kids_profile
0,6383.833644,80,6.565820,59.771942,7,0,0,1,mobile,premium,West,1
1,1370.968822,23,1.019404,48.541942,7,1,0,2,desktop,free,Northeast,0
2,12024.576599,157,0.914584,44.892548,10,1,1,3,tv,basic,Northeast,0


In [32]:
simulated_users.columns

Index(['TotalWatchTime', 'UniqueMovies', 'TenureMonths', 'Age', 'Occupation',
       'treatment', 'renewed', 'UserID', 'device_type', 'subscription_tier',
       'region', 'has_kids_profile'],
      dtype='object')

In [33]:
simulated_users.dtypes

,0
TotalWatchTime,float64
UniqueMovies,int64
TenureMonths,float64
Age,float64
Occupation,int64
treatment,int64
renewed,int64
UserID,int64
device_type,object
subscription_tier,object


In [34]:
simulated_users['has_kids_profile'] = simulated_users['has_kids_profile'].astype('bool')

In [35]:
simulated_users.dtypes

,0
TotalWatchTime,float64
UniqueMovies,int64
TenureMonths,float64
Age,float64
Occupation,int64
treatment,int64
renewed,int64
UserID,int64
device_type,object
subscription_tier,object


In [36]:
# Explicitly exclude columns not intended as features
non_feature_cols = ['UserID', 'treatment', 'renewed']
feature_df = simulated_users.drop(columns=non_feature_cols)

# Automatically detect feature types
numeric_features = feature_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = feature_df.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()

print("✅ Numeric features:", numeric_features)
print("✅ Categorical features:", categorical_features)

# Now define feature matrix
X = simulated_users[numeric_features + categorical_features]
T = simulated_users['treatment'].values
Y = simulated_users['renewed'].values


✅ Numeric features: ['TotalWatchTime', 'UniqueMovies', 'TenureMonths', 'Age', 'Occupation']
✅ Categorical features: ['device_type', 'subscription_tier', 'region', 'has_kids_profile']


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Train/test split
X_train, X_test, T_train, T_test, Y_train, Y_test = train_test_split(
    X, T, Y, test_size=0.2, random_state=42
)



In [ ]:
# Define preprocessor only for those base learners that need preprocessing, like logistic regression, svm but not for trees
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
])

# Transform features
X_train_proc = preprocessor.fit_transform(X_train)
X_test_proc = preprocessor.transform(X_test)

In [ ]:
# define preprocessor of categorical features for random forest, gradient boosting and XGBoost
from sklearn.preprocessing import LabelEncoder

# Copy features to avoid changing original
tree_features = simulated_users.copy()

# Encode all categorical columns
label_encoders = {}

for col in categorical_features:
    le = LabelEncoder()
    tree_features[col] = le.fit_transform(tree_features[col])
    label_encoders[col] = le  # Optional: save encoders for inverse_transform later


In [ ]:
tree_features.head()


,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation,treatment,renewed,UserID,device_type,subscription_tier,region,has_kids_profile
0,7440.027920,80,7.239655,59.614629,7,0,0,1,1,2,3,1
1,1493.701456,23,1.730815,49.277609,7,1,0,2,0,1,1,0
2,10790.226372,157,1.185210,46.291427,10,1,0,3,3,0,1,0
3,1755.714462,22,0.953775,39.189768,7,1,0,4,3,0,3,1
4,27024.149372,361,29.990638,61.782170,6,0,0,5,1,0,2,0


In [ ]:
tree_features.dtypes

,0
TotalWatchTime,float64
UniqueMovies,int64
TenureMonths,float64
Age,float64
Occupation,int64
treatment,int64
renewed,int64
UserID,int64
device_type,int64
subscription_tier,int64


In [ ]:
non_feature_cols = ['UserID', 'treatment', 'renewed']
X_tree = tree_features.drop(columns=non_feature_cols)
T = simulated_users['treatment'].values
Y = simulated_users['renewed'].values

In [ ]:
from sklearn.model_selection import train_test_split

X_train_tree, X_test_tree, T_train, T_test, Y_train, Y_test = train_test_split(
    X_tree, T, Y, test_size=0.2, random_state=42
)

In [ ]:
X_tree.dtypes

,0
TotalWatchTime,float64
UniqueMovies,int64
TenureMonths,float64
Age,float64
Occupation,int64
device_type,int64
subscription_tier,int64
region,int64
has_kids_profile,int64


In [41]:
simulated_users.columns

Index(['TotalWatchTime', 'UniqueMovies', 'TenureMonths', 'Age', 'Occupation',
       'treatment', 'renewed', 'UserID', 'device_type', 'subscription_tier',
       'region', 'has_kids_profile'],
      dtype='object')

In [38]:
 np.random.choice(simulated_users['UserID'], size=50000, replace=True)

array([  163, 10721,   565, ...,  6774,  6758,  6616])

In [40]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [42]:
 pd.date_range(start='2023-01-01', periods=50000, freq='h')

'H' is deprecated and will be removed in a future version, please use 'h' instead.


DatetimeIndex(['2023-01-01 00:00:00', '2023-01-01 01:00:00',
               '2023-01-01 02:00:00', '2023-01-01 03:00:00',
               '2023-01-01 04:00:00', '2023-01-01 05:00:00',
               '2023-01-01 06:00:00', '2023-01-01 07:00:00',
               '2023-01-01 08:00:00', '2023-01-01 09:00:00',
               ...
               '2028-09-13 22:00:00', '2028-09-13 23:00:00',
               '2028-09-14 00:00:00', '2028-09-14 01:00:00',
               '2028-09-14 02:00:00', '2028-09-14 03:00:00',
               '2028-09-14 04:00:00', '2028-09-14 05:00:00',
               '2028-09-14 06:00:00', '2028-09-14 07:00:00'],
              dtype='datetime64[ns]', length=50000, freq='h')

In [45]:
#now simulate another table for user logs
user_activity_logs = pd.DataFrame({
    'UserID': np.random.choice(simulated_users['UserID'], size=50000, replace=True),
    'timestamp': pd.date_range(start='2023-01-01', periods=50000, freq='h'),
    'action': np.random.choice(['watch', 'pause', 'stop', 'open_app', 'click_promo'], size=50000),
    'device_type': np.random.choice(['mobile', 'tv', 'desktop'], size=50000)
})

In [46]:
user_activity_logs.head(3)

,UserID,timestamp,action,device_type
0,14064,2023-01-01 00:00:00,watch,desktop
1,19633,2023-01-01 01:00:00,watch,desktop
2,11688,2023-01-01 02:00:00,open_app,desktop


In [47]:
simulated_users.head(3)


,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation,treatment,renewed,UserID,device_type,subscription_tier,region,has_kids_profile
0,6383.833644,80,6.565820,59.771942,7,0,0,1,mobile,premium,West,True
1,1370.968822,23,1.019404,48.541942,7,1,0,2,desktop,free,Northeast,False
2,12024.576599,157,0.914584,44.892548,10,1,1,3,tv,basic,Northeast,False


In [48]:
#What’s the overall renewal rate for the dataset?
simulated_users['renewed'].mean()

0.24535

In [49]:
#What’s the renewal rate by treatment group?
simulated_users.groupby('treatment')['renewed'].mean()

,renewed
treatment,
0,0.205479
1,0.285815


In [54]:
#What’s the uplift (difference in renewal) between treatment and control?
renewal_by_group = simulated_users.groupby('treatment')['renewed'].mean()
uplift = renewal_by_group.loc[1] - renewal_by_group.loc[0]
print(f"Uplift: {uplift*100:.2f}%")


Uplift: 8.03%


In [55]:
simulated_users.head(2)

,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation,treatment,renewed,UserID,device_type,subscription_tier,region,has_kids_profile
0,6383.833644,80,6.565820,59.771942,7,0,0,1,mobile,premium,West,True
1,1370.968822,23,1.019404,48.541942,7,1,0,2,desktop,free,Northeast,False


In [65]:
#Which device type has the highest renewal rate?
renewal_by_device= simulated_users.groupby('device_type')['renewed'].mean().sort_values(ascending = False).reset_index()
renewal_by_device

,device_type,renewed
0,tablet,0.260355
1,tv,0.248457
2,desktop,0.246058
3,mobile,0.238853


In [69]:
print(f"The device with the hightest renewal rate is {renewal_by_device.loc[0,'device_type']}")

The device with the hightest renewal rate is tablet


In [70]:
simulated_users.head(2)

,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation,treatment,renewed,UserID,device_type,subscription_tier,region,has_kids_profile
0,6383.833644,80,6.565820,59.771942,7,0,0,1,mobile,premium,West,True
1,1370.968822,23,1.019404,48.541942,7,1,0,2,desktop,free,Northeast,False


In [73]:
#How many users are in each treatment group per region?
simulated_users.groupby(['treatment', 'region'])['UserID'].nunique()

treatment  region   
0          Midwest      2562
           Northeast    2540
           South        2490
           West         2482
1          Midwest      2492
           Northeast    2511
           South        2484
           West         2439
Name: UserID, dtype: int64

In [75]:
simulated_users.groupby(['treatment', 'region'])['UserID'].count().unstack()

region,Midwest,Northeast,South,West
treatment,,,,
0,2562,2540,2490,2482
1,2492,2511,2484,2439


In [85]:
#Create a new column: renewal rate per subscription tier
simulated_users['renewal_by_tier']  =simulated_users.groupby('subscription_tier')['renewed'].transform('mean')
#simulated_users['renewal_by_tier'] = simulated_users.groupby('subscription_tier')['renewed'].mean()
simulated_users.head(2)




,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation,treatment,renewed,UserID,device_type,subscription_tier,region,has_kids_profile,renewal_by_tier
0,6383.833644,80,6.565820,59.771942,7,0,0,1,mobile,premium,West,True,0.243622
1,1370.968822,23,1.019404,48.541942,7,1,0,2,desktop,free,Northeast,False,0.234580


In [86]:
simulated_users.columns

Index(['TotalWatchTime', 'UniqueMovies', 'TenureMonths', 'Age', 'Occupation',
       'treatment', 'renewed', 'UserID', 'device_type', 'subscription_tier',
       'region', 'has_kids_profile', 'renewal_by_tier'],
      dtype='object')

In [90]:
#What’s the average watch time and age by treatment group?
simulated_users.groupby('treatment')[['TotalWatchTime', 'Age']].mean().round(2)


,TotalWatchTime,Age
treatment,,
0,12871.43,43.36
1,12076.54,43.32


In [92]:
simulated_users.groupby('treatment')[['TotalWatchTime', 'Age']].mean()\
.applymap(lambda x: f'{x:.2f}')

DataFrame.applymap has been deprecated. Use DataFrame.map instead.


,TotalWatchTime,Age
treatment,,
0,12871.43,43.36
1,12076.54,43.32


In [93]:
simulated_users.head(2)

,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation,treatment,renewed,UserID,device_type,subscription_tier,region,has_kids_profile,renewal_by_tier
0,6383.833644,80,6.565820,59.771942,7,0,0,1,mobile,premium,West,True,0.243622
1,1370.968822,23,1.019404,48.541942,7,1,0,2,desktop,free,Northeast,False,0.234580


In [94]:
#For each user, create a flag if they’re in the top 10% of watch time
simulated_users['top_10_percent'] = simulated_users['TotalWatchTime'] >= simulated_users['TotalWatchTime'].quantile(0.9)
simulated_users.head(2)



,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation,treatment,renewed,UserID,device_type,subscription_tier,region,has_kids_profile,renewal_by_tier,top_10_percent
0,6383.833644,80,6.565820,59.771942,7,0,0,1,mobile,premium,West,True,0.243622,False
1,1370.968822,23,1.019404,48.541942,7,1,0,2,desktop,free,Northeast,False,0.234580,False


In [100]:
#Show uplift by subscription tier
simulated_users.groupby(['subscription_tier', 'treatment'])['renewed'].mean()

subscription_tier  treatment
basic              0            0.208706
                   1            0.293834
free               0            0.206738
                   1            0.261893
premium            0            0.199057
                   1            0.288475
Name: renewed, dtype: float64

In [109]:
tier_renewal = simulated_users.groupby(['subscription_tier', 'treatment'])['renewed'].mean().unstack()
tier_renewal

treatment,0,1
subscription_tier,,
basic,0.208706,0.293834
free,0.206738,0.261893
premium,0.199057,0.288475


In [110]:
tier_renewal.columns

Index([0, 1], dtype='int64', name='treatment')

In [111]:
tier_renewal[1]

,1
subscription_tier,
basic,0.293834
free,0.261893
premium,0.288475


In [112]:
tier_renewal['uplift'] = tier_renewal[1] - tier_renewal[0]
tier_renewal

treatment,0,1,uplift
subscription_tier,,,
basic,0.208706,0.293834,0.085128
free,0.206738,0.261893,0.055155
premium,0.199057,0.288475,0.089418


In [117]:
tier_renewal.columns.dtype

dtype('O')

In [118]:
tier_renewal.columns

Index([0, 1, 'uplift'], dtype='object', name='treatment')

In [98]:
tier_group = simulated_users.groupby(['subscription_tier', 'treatment'])['renewed'].mean().unstack()
tier_group['uplift'] = tier_group[1] - tier_group[0]
tier_group

treatment,0,1,uplift
subscription_tier,,,
basic,0.208706,0.293834,0.085128
free,0.206738,0.261893,0.055155
premium,0.199057,0.288475,0.089418


In [120]:
#Bin users into 5 segments based on propensity score and compute renewal rate
simulated_users.head(2)
simulated_users['propensity_bin'] = pd.qcut(simulated_users['propensity_score'], q=5, labels=False)
simulated_users.groupby(['propensity_bin', 'treatment'])['renewed'].mean().unstack()

KeyError: 'propensity_score'

In [121]:
simulated_users.head(2)

,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation,treatment,renewed,UserID,device_type,subscription_tier,region,has_kids_profile,renewal_by_tier,top_10_percent
0,6383.833644,80,6.565820,59.771942,7,0,0,1,mobile,premium,West,True,0.243622,False
1,1370.968822,23,1.019404,48.541942,7,1,0,2,desktop,free,Northeast,False,0.234580,False


In [122]:
user_activity_logs.head(2)

,UserID,timestamp,action,device_type
0,14064,2023-01-01 00:00:00,watch,desktop
1,19633,2023-01-01 01:00:00,watch,desktop


In [123]:
# Join user info with activity logs
user_merged = simulated_users.merge(user_activity_logs, on = 'UserID', how = 'left')
user_merged.head()


,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation,treatment,renewed,UserID,device_type_x,subscription_tier,region,has_kids_profile,renewal_by_tier,top_10_percent,timestamp,action,device_type_y
0,6383.833644,80,6.565820,59.771942,7,0,0,1,mobile,premium,West,True,0.243622,False,2024-08-09 09:00:00,open_app,tv
1,1370.968822,23,1.019404,48.541942,7,1,0,2,desktop,free,Northeast,False,0.234580,False,2023-09-06 18:00:00,stop,desktop
2,12024.576599,157,0.914584,44.892548,10,1,1,3,tv,basic,Northeast,False,0.250568,False,NaT,NaN,NaN
3,1036.000000,22,1.606146,37.677855,7,1,1,4,tv,basic,West,True,0.250568,False,2023-01-13 03:00:00,open_app,desktop
4,1036.000000,22,1.606146,37.677855,7,1,1,4,tv,basic,West,True,0.250568,False,2024-03-20 21:00:00,open_app,desktop


In [126]:
merged2 = user_activity_logs.merge(simulated_users[['UserID', 'region' , 'subscription_tier']], on = 'UserID', how = 'left')
merged2.sort_values('UserID').head()

,UserID,timestamp,action,device_type,region,subscription_tier
14073,1,2024-08-09 09:00:00,open_app,tv,West,premium
5970,2,2023-09-06 18:00:00,stop,desktop,Northeast,free
291,4,2023-01-13 03:00:00,open_app,desktop,West,basic
10677,4,2024-03-20 21:00:00,open_app,desktop,West,basic
39499,4,2027-07-04 19:00:00,pause,mobile,West,basic


In [130]:
# ✅ 2. How many total actions per user?
merged2.groupby('UserID')['action'].count().reset_index(name = 'action_count')



,UserID,action_count
0,1,1
1,2,1
2,4,5
3,5,4
4,6,3
...,...,...
18337,19994,2
18338,19995,3
18339,19996,1
18340,19998,4


In [142]:
user_activity_logs.groupby(['UserID'])['action'].value_counts().reset_index(name='count')

,UserID,action,count
0,1,open_app,1
1,2,stop,1
2,4,open_app,3
3,4,pause,1
4,4,watch,1
...,...,...,...
39352,19995,watch,1
39353,19996,watch,1
39354,19998,watch,3
39355,19998,open_app,1


In [143]:
user_activity_logs.groupby(['UserID'])['action'].value_counts().reset_index(name='count').drop_duplicates('UserID')

,UserID,action,count
0,1,open_app,1
1,2,stop,1
2,4,open_app,3
5,5,open_app,2
8,6,open_app,1
...,...,...,...
39348,19994,open_app,1
39350,19995,click_promo,1
39353,19996,watch,1
39354,19998,watch,3


In [134]:
user_activity_logs.groupby(['UserID', 'action']).size().reset_index(name='count')

,UserID,action,count
0,1,open_app,1
1,2,stop,1
2,4,open_app,3
3,4,pause,1
4,4,watch,1
...,...,...,...
39352,19995,watch,1
39353,19996,watch,1
39354,19998,open_app,1
39355,19998,watch,3


In [139]:
# 3 What’s the most common action for each user?
user_activity_logs.groupby(['UserID', 'action']).size().reset_index(name='count')\
    .sort_values(['UserID', 'count'], ascending=[True, False]).drop_duplicates('UserID')

,UserID,action,count
0,1,open_app,1
1,2,stop,1
2,4,open_app,3
6,5,open_app,2
8,6,open_app,1
...,...,...,...
39348,19994,open_app,1
39350,19995,click_promo,1
39353,19996,watch,1
39355,19998,watch,3


In [144]:
user_activity_logs.head(2)

,UserID,timestamp,action,device_type
0,14064,2023-01-01 00:00:00,watch,desktop
1,19633,2023-01-01 01:00:00,watch,desktop


In [145]:
merged2.head()

,UserID,timestamp,action,device_type,region,subscription_tier
0,14064,2023-01-01 00:00:00,watch,desktop,West,free
1,19633,2023-01-01 01:00:00,watch,desktop,Northeast,premium
2,11688,2023-01-01 02:00:00,open_app,desktop,South,basic
3,9143,2023-01-01 03:00:00,open_app,desktop,Midwest,premium
4,6076,2023-01-01 04:00:00,click_promo,desktop,South,basic


In [151]:
simulated_users.head(2)

,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation,treatment,renewed,UserID,device_type,subscription_tier,region,has_kids_profile,renewal_by_tier,top_10_percent
0,6383.833644,80,6.565820,59.771942,7,0,0,1,mobile,premium,West,True,0.243622,False
1,1370.968822,23,1.019404,48.541942,7,1,0,2,desktop,free,Northeast,False,0.234580,False


In [152]:
user_activity_logs.head(2)

,UserID,timestamp,action,device_type
0,14064,2023-01-01 00:00:00,watch,desktop
1,19633,2023-01-01 01:00:00,watch,desktop


In [161]:
 # 4 Find average number of 'watch' actions by subscription tier
watch_logs = user_activity_logs[user_activity_logs['action'] == 'watch']
merged = watch_logs.merge(simulated_users[['UserID', 'subscription_tier']], on='UserID', how='left')
merged.head()
#merged.groupby('subscription_tier')['UserID'].count() / merged['UserID'].nunique()

,UserID,timestamp,action,device_type,subscription_tier
0,14064,2023-01-01 00:00:00,watch,desktop,free
1,19633,2023-01-01 01:00:00,watch,desktop,premium
2,8804,2023-01-01 07:00:00,watch,mobile,premium
3,17477,2023-01-01 11:00:00,watch,tv,premium
4,7220,2023-01-01 16:00:00,watch,tv,premium


In [166]:
merged.groupby('subscription_tier')['UserID'].count()

,UserID
subscription_tier,
basic,5067
free,1937
premium,2876


In [164]:
merged.groupby('subscription_tier')['UserID'].nunique()

,UserID
subscription_tier,
basic,4010
free,1533
premium,2262


In [167]:
merged['UserID'].nunique()

7805

In [165]:
merged.shape[0]

9880

In [168]:
merged.groupby('subscription_tier')['UserID'].count()/merged['UserID'].nunique()

,UserID
subscription_tier,
basic,0.649199
free,0.248174
premium,0.368482


In [172]:
user_activity_logs[user_activity_logs['action'] == 'click_promo'].head(3)

,UserID,timestamp,action,device_type
4,6076,2023-01-01 04:00:00,click_promo,desktop
8,16662,2023-01-01 08:00:00,click_promo,mobile
10,7924,2023-01-01 10:00:00,click_promo,desktop


In [173]:
simulated_users[simulated_users['renewed'] == 0].head(3)

,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation,treatment,renewed,UserID,device_type,subscription_tier,region,has_kids_profile,renewal_by_tier,top_10_percent
0,6383.833644,80,6.565820,59.771942,7,0,0,1,mobile,premium,West,True,0.243622,False
1,1370.968822,23,1.019404,48.541942,7,1,0,2,desktop,free,Northeast,False,0.234580,False
4,26537.629491,361,30.929612,60.148336,6,0,0,5,mobile,basic,South,False,0.250568,False


In [176]:
# Which users clicked on a promo but did NOT renew?
merged3 = user_activity_logs[user_activity_logs['action'] == 'click_promo']\
.merge(simulated_users[simulated_users['renewed'] == 0][['UserID', 'renewed']], on = 'UserID', how = 'inner')
merged3.head()

,UserID,timestamp,action,device_type,renewed
0,6076,2023-01-01 04:00:00,click_promo,desktop,0
1,14859,2023-01-01 15:00:00,click_promo,desktop,0
2,14217,2023-01-01 18:00:00,click_promo,tv,0
3,16503,2023-01-01 20:00:00,click_promo,mobile,0
4,11676,2023-01-02 05:00:00,click_promo,desktop,0


In [177]:
merged3[['UserID']].drop_duplicates()

,UserID
0,6076
1,14859
2,14217
3,16503
4,11676
...,...
7558,13610
7559,8101
7561,11898
7562,9784
